In [90]:
import requests
from urllib.parse import urlencode

base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = "https://disk.yandex.ru/d/V1sJpR-SUJ_b8A"

final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url)
download_url = response.json()['href']

download_response = requests.get(download_url)
with open('dataset.xlsx', 'wb') as f:
    f.write(download_response.content)


In [91]:
from pathlib import Path
import pandas as pd

xlsx_path = "dataset.xlsx"
if not xlsx_path:
    raise FileNotFoundError("xlsx файл не найден")
print("Найден XLSX:", xlsx_path)

df = pd.read_excel(xlsx_path, sheet_name=0, header=[0,1])
print("Данные загружены в df")


Найден XLSX: dataset.xlsx
Данные загружены в df


In [92]:
import re
from collections import Counter, defaultdict

def clean(s):
    if s is None: return ""
    s = str(s).replace("\n"," ").replace("\xa0"," ").strip()
    s = re.sub(r"\s+", " ", s)
    return s

top_vals = [clean(t) for t,_ in df.columns]
top_vals = [("" if t.lower().startswith("unnamed") else t) for t in top_vals]
top_counts = Counter([t for t in top_vals if t])

seen = defaultdict(int)
new_cols = []
for (t, s) in df.columns:
    t = clean(t)
    t = "" if t.lower().startswith("unnamed") else t
    s = clean(s)

    if t:
        seen[t] += 1
        if top_counts[t] > 1:
            t = f"{t} {seen[t]}"
        else:
            t = t

    new_cols.append((t, s))

df.columns = pd.MultiIndex.from_tuples(new_cols)

def flatten(t, s, sep="_"):
    if t == "" or t is None:
        name = s
    else:
        name = f"{t}{sep}{s}"

    name = name.strip()
    name = re.sub(r"\s+", "_", name)
    name = re.sub(r"[/\\.;:]+", "_", name)
    name = re.sub(r"[,-]+", "", name)
    name = name.strip("_")
    return name

df.columns = [flatten(t, s) for (t, s) in df.columns]

assert not pd.Series(df.columns).duplicated().any(), "Остались дубли имён"
df.head()


,Наименование,Изготовитель,Номер_свидетельства_и_дата,Этикетка,Происхождение,Противопоказания,Срок_годности,Год_регистрации,Количество_групп_компонентов,Пищевые_вещества_1_Витамины_витаминоподобные_вещества_и_коферменты,Пищевые_вещества_2_Макро_и_микроэлементы,Пищевые_вещества_3_Жиры_жироподобные_вещества_и_их_производные,Пищевые_вещества_4_Белки_пептиды_аминокислоты_нуклеиновые_кислоты,Минорные_компоненты_растений_1_Фенольные_соединения,Минорные_компоненты_растений_2_Алкалоиды,Пробиотики_в_монокультурах_и_ассоциациях_Пробиотические_микроорганизмы,Пищевые_вещества_5_Углеводы_и_продукты_их_переработки,Минорные_компоненты_растений_3_Сапонины,Минорные_компоненты_растений_4_Терпеноиды,Минорные_компоненты_растений_5_Естественные_метаболиты_и_стимуляторы_метаболизма,Минорные_компоненты_растений_6_Гидроксикоричные_кислоты,Минорные_компоненты_растений_7_Ферменты,Минорные_компоненты_растений_8_Дубильные_вещества,Минеральные_и_минералоорганические_природные_субстанции_Цеолиты_Гуминовые_кислоты,Система_органов_1_для_беременных_кормящих_и_планирующих_беременность,Система_органов_2_костномышечная_сиситема,Система_органов_3_нервная_система,Система_органов_4_иммунная_система,Система_органов_5_пищеварительный_тракт_и_обмен_веществ,Система_органов_6_мочеполовая_система,Система_органов_7_дерматологические_БАД,Система_органов_8_органы_чувств,Система_органов_9_сердечнососудистая_система,Система_органов_10_противоопухолевые_БАД,Система_органов_11_противопаразитарные_БАД,Система_органов_12_дыхательная_система,Система_органов_13_кровь_и_система_кроветворения,Система_органов_14_противомикробные_БАД,Система_органов_15_Форма_выпуска,Система_органов_16_Продолжительность_приема,Система_органов_17_Происхождение_(природное_синтетическое),Система_органов_18_Сырье_(растительное_животное_биологическое),Группа_населения_1_предназначен_для_детей,Группа_населения_2_возраст_детей,Группа_населения_3_предназначен_для_взрослых,Группа_населения_4_пол,Группа_населения_5_пожилые
0,"биологически активная добавка к пище ""ВитаФин"" (""VitaFin"") (таблетки массой 700 мг)","Ханкинтатукку Оу"" (""Hankintatukku Oy""), Lehtolankatu 18, 03600 Karkkila (Финляндия)",RU.77.99.11.003.Е.000894.09.10 от 10.09.2010,"Рекомендации по применению: взрослым по 1-2 таблетки в день во время еды. Продолжительность приема - 1-2 месяца. При необходимости прием можно повторить. Противопоказания: индивидуальная непереносимость компонентов, беременность, кормление грудью. Перед применением рекомендуется проконсультироваться с врачом. Лицам с заболеваниями щитовидной микроэлементыы рекомендуется проконсультироваться с врачом-эндокринологом. Срок годности - 3 года. Хранить в сухом, недоступном для детей месте, при температуре не выше 25°С.",иностранное,"индивидуальная непереносимость компонентов, беременность, кормление грудью",3 года,2010,3,вит,элементы,NaN,NaN,фенольн,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,иммунитет,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,таблетки,1-2 месяца,Природное (растительное),"Экстракта зелёного чая (Camellia sinensis - Камелия китайская), экстракт кожицы винограда (Винный виноград - Vitis Vinifera",NaN,NaN,взрослые,NaN,NaN
1,"биологически активная добавка к пище ""Гепаплант"" (""Hepaplant"") (таблетки массой 1000 мг)","Ханкинтатукку Оу"" (""Hankintatukku Oy""), Lehtolankatu 18, 03600 Karkkila (Финляндия)",RU.77.99.11.003.Е.000895.09.10 от 10.09.2010,"Рекомендации по применению: взрослым по 1 таблетке 2-3 раза в день во время еды. Продолжительность приема - 1-2 месяца. При необходимости прием можно повторить. Противопоказания: индивидуальная непереносимость компонентов, беременность, кормление грудью. Перед применением рекомендуется проконсультироваться с врачом. Срок годности - 3 года. Хранить в сухом, недоступном для детей месте, при температуре не выше 25°С.",иностранное,"индивидуальная непереносимость компонентов, беременность, кормление грудью",3 года,2010,много,вит,NaN,пнжк,NaN,фенольн,NaN,NaN,NaN,NaN,NaN,NaN,гидроксикор,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [93]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


Создаем 2 новых столбца:
- Рекомендации по применению
- Продолжительность приема

Информацию для них берем из столбца Этикетка, затем отчищаем оттуда взятую инфу.

In [94]:
df['Рекомендации_по_применению'] = 'a'
df['Продолжительность_приема'] = 'b'

for row in range(len(df)):
  string = str(df.at[row, 'Этикетка'])

  dot = string.find(".")+1
  str_for_et = str(df.at[row, 'Этикетка'])[string.find(".", dot)+1::]

  value_1 = string[string.find("Рекомендации_по_применению:"):dot]
  df.at[row, 'Рекомендации_по_применению'] = value_1

  value_2 = string[string.find("Продолжительность_приема"):string.find(".", dot)]
  df.at[row, 'Продолжительность_приема'] = value_2

  df.at[row, 'Этикетка'] = str_for_et



Срок годности преобразуем из лет в месяцы

Посмотрим уникальные значения до изменений

In [95]:
print(df['Срок_годности'].unique())

['3 года' '2 года' '1 год' '4 года' '1,5 года' '1 год, 2 года' '8 месяцев'
 '2 года, 1 год' '2,5 месяца' '5 лет' '1 месяц' '2 месяца' '19 месяцев'
 '28 месяцев' '1 год, 2 месяца' '3,5 года' '6 месяцев' '2,5 года'
 '32 месяца' '21 месяц' '14 месяцев' '3 месяца' '2 года, 1,5 года'
 '15 суток' '11 месяцев' '20 месяцев' '25 месяцев']


In [98]:
bad_val = {"1 год, 2 года" : 24,
           "1 год, 2 месяца": 14,
           "1,5 года": 18,
           "15 суток": 0.5,
           "2 года": 24,
           "2 года, 1 год": 24,
           "2 года, 1,5 года": 24,
           "2,5 года": 30,
           "3 года": 36,
           "3,5 года": 42,
           "4 года": 48,
           "5 лет": 60,
           "1 год": 12
           }
for row in range(len(df)):
  val = str(df.at[row, 'Срок_годности'])
  if val in bad_val.keys():
    df.at[row, 'Срок_годности'] = bad_val[val]
  else:
    if ('меся' in str(df.at[row, 'Срок_годности'])):
      string = str(df.at[row, 'Срок_годности'])
      str_res = string[:string.index(" ")]
      if (str_res != ''):
        if (',' in str(df.at[row, 'Срок_годности'])):
          str_res = str_res.replace(',', '.')
        df.at[row, 'Срок_годности'] = float(str_res)

0       36
1       36
2       24
3       24
4       24
        ..
4218    36
4219    36
4220    36
4221    24
4222    36
Name: Срок_годности, Length: 4223, dtype: object


Посмотрим уникальные значения после (Все преобразовалось)

In [99]:
print(df['Срок_годности'].unique())

[36 24 12 48 18 8.0 2.5 60 1.0 2.0 19.0 28.0 14 42 6.0 30 32.0 21.0 3.0
 0.5 11.0 20.0 25.0]


In [101]:
print(df.columns)

Index(['Наименование', 'Изготовитель', 'Номер_свидетельства_и_дата',
       'Этикетка', 'Происхождение', 'Противопоказания', 'Срок_годности',
       'Год_регистрации', 'Количество_групп_компонентов',
       'Пищевые_вещества_1_Витамины_витаминоподобные_вещества_и_коферменты',
       'Пищевые_вещества_2_Макро_и_микроэлементы',
       'Пищевые_вещества_3_Жиры_жироподобные_вещества_и_их_производные',
       'Пищевые_вещества_4_Белки_пептиды_аминокислоты_нуклеиновые_кислоты',
       'Минорные_компоненты_растений_1_Фенольные_соединения',
       'Минорные_компоненты_растений_2_Алкалоиды',
       'Пробиотики_в_монокультурах_и_ассоциациях_Пробиотические_микроорганизмы',
       'Пищевые_вещества_5_Углеводы_и_продукты_их_переработки',
       'Минорные_компоненты_растений_3_Сапонины',
       'Минорные_компоненты_растений_4_Терпеноиды',
       'Минорные_компоненты_растений_5_Естественные_метаболиты_и_стимуляторы_метаболизма',
       'Минорные_компоненты_растений_6_Гидроксикоричные_кислоты',
      

Также меняем столбец возраст детей. Меняем года на месяцы
Будет указано одно значение (12 например). Оно будет означать, что для детей от 12 месяцев.

In [103]:
bad_val = {"от 11 лет" : 132,
           "от 12 лет": 144,
           "от 14 лет": 168,
           "от 3 месяцев": 3,
           "от 7 лет": 84,
           "с рождения": 0,
           "от 1,5 лет": 24,
           "от 3 лет": 36,
           "от 4 лет": 48,
           "от 5 лет": 60,
           "от 1 года": 12
           }
for i in range(len(df)):
  val = str(df.at[i, 'Группа_населения_2_возраст_детей'])
  if val in bad_val.keys():
    df.at[i, 'Группа_населения_2_возраст_детей'] = bad_val[val]
print(df['Группа_населения_2_возраст_детей'].unique())

[nan 132 84 0 168 36 48 144 3 12 24 60]
